In [1]:
import time
import math
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import dataset
import cv2

from sklearn.metrics import confusion_matrix
from datetime import timedelta

import tensorflow as tf
import dataset

import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
num_channels = 3
img_size = 244
img_size_flat = img_size * img_size * num_channels

img_shape = (img_size, img_size)

classes = ['empty', 'full']
num_classes = len(classes)

batch_size = 32

validation_size = .2

train_path = './dataset/'
test_path = './test/'
checkpoint_dir = './models/'

In [3]:
image_shape = [None, img_size, img_size, num_channels]

In [4]:
data = dataset.read_train_sets(train_path, img_size, classes, validation_size=validation_size)
test_images, test_ids = dataset.read_test_set(test_path, img_size)

Reading training images
Loading empty files (Index: 0)
Loading full files (Index: 1)
Reading test images


In [5]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(test_images)))
print("- Validation-set:\t{}".format(len(data.valid.labels)))

Size of:
- Training-set:		2117
- Test-set:		5
- Validation-set:	529


In [6]:
def flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat #, num_features

## MobileNetV2

In [7]:
def _conv_block(inputs, filters, kernel, strides):
    
    x = tf.nn.conv2d(filters, kernel, padding='same', strides=strides)(inputs)
    x = tf.nn.batch_normalization()(x)
    return relu6(x)


def _bottleneck(inputs, filters, kernel, t, s, r=False):
    
    tchannel = inputs.shape[-1] * t
    x = _conv_block(inputs, tchannel, (1, 1), (1, 1))
    x = tf.nn.depthwise_conv2d(kernel, strides=(s, s), padding='same')(x)
    x = tf.nn.batch_normalization()(x)
    x = tf.nn.relu6(x)
    
    x = tf.nn.conv2d(filters, (1, 1), strides=(1, 1), padding='same')(x)
    x = tf.nn.batch_normalization()(x)

    if r:
        x = x = tf.math.add(x, inputs)
    return x


def _inverted_residual_block(inputs, filters, kernel, t, strides, n):
    
    x = _bottleneck(inputs, filters, kernel, t, strides)
    
    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, 1, True)

    return x

def MobileNetV2(inputs):
    """MobileNetv2
    This function defines a MobileNetv2 architecture.
    # Arguments
        input_shape: An integer or tuple/list of 3 integers, shape
            of input tensor.
        k: Integer, number of classes.
        plot_model: Boolean, whether to plot model architecture or not
    # Returns
        MobileNetv2 model.
    """

#     x = tf.placeholder("float", image_shape)
#     inputs = Input(shape=input_shape, name='input')
    x = _conv_block(inputs, 32, (3, 3), strides=(2, 2))

    x = _inverted_residual_block(x, 16, (3, 3), t=1, strides=1, n=1)
    x = _inverted_residual_block(x, 24, (3, 3), t=6, strides=2, n=2)
    x = _inverted_residual_block(x, 32, (3, 3), t=6, strides=2, n=3)
    x = _inverted_residual_block(x, 64, (3, 3), t=6, strides=2, n=4)
    x = _inverted_residual_block(x, 96, (3, 3), t=6, strides=1, n=3)
    x = _inverted_residual_block(x, 160, (3, 3), t=6, strides=2, n=3)
    x = _inverted_residual_block(x, 320, (3, 3), t=6, strides=1, n=1)

    x = _conv_block(x, 1280, (1, 1), strides=(1, 1))
#     x = flatten_layer(x)
    x = tf.reshape((1, 1, 1280))(x)
    x = tf.nn.dropout(0.3, name='Dropout')(x)
    x = tf.nn.conv2d(k, (1, 1), padding='same')(x)
    output = tf.nn.softmax('softmax', name='final_activation')(x)
#     output = tf.reshape((k,), name='output')(x)
    
#     model = tf.keras.models.Model(inputs, output)
#     model.summary()
#     if plot_model:
#         tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

    return output

def trashcan_net(inputs):
    
    x = MobileNetV2(inputs)
    x = tf.nn.avg_pool(ksize=(7, 7))(x)
    x = flatten_layer(x)
    x = tf.compat.v1.layers.dense(128)(x)
    x = tf.nn.relu6(x)
    x = tf.nn.dropout(0.5)(x)
    x = tf.compat.v1.layers.dense(2)(x)
    output = tf.nn.softmax()(x)
    
    return output

In [8]:
grad_log = []

epochs = 10 

for epoch in range(epochs): 
    print(f'\nНачинаем эпоху {epoch}') 

    for step in range(int(len(data.train.labels)/batch_size)):
        
        with tf.GradientTape() as tape: 
            
            x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)
            
            print(x_batch.shape)
            
            preds = trashcan_net(x_batch)
            
            
            loss_value = loss_fn(y_true_batch, preds)
            
            grads = tape.gradient(loss_value, trashcan_net.trainable_weights) 
            g_g = []

            # пишем логи для сохранения значений градиента и веса по одной цепи 
            for g_s in grads:
                # допишем логи значений градиента в зависимости от размера тензора градиента
                # if len(g_s.numpy().shape) == 1:
                #     g_g.append(g_s.numpy()[0])
                if len(g_s.numpy().shape) == 2:
                    g_g.append(g_s.numpy()[0, 0]) 

                
        # добавляем текущие логи по слоям к общей записи
        grad_log.append(g_g)

        # Выполним один шаг градиентного спуска,
        # обновив значение переменных минимизирующих потери
        optimizer.apply_gradients(zip(grads, trashcan_net.trainable_weights)) 

        # Пишем лог каждые 200 шагов
        if step % 200 == 0:
            print(f'Эпоха {epoch + 1}/{epochs}', end='. ')
            print(f'Шаг {step}. Лосс на обучении (для одного батча) на шаге: {loss_value}') 
            print(f'Уже увидели: {(step + 1) * batch_size} примеров')


Начинаем эпоху 0
(32, 244, 244, 3)


AttributeError: 'int' object has no attribute 'shape'

In [9]:
def trashcan_det(inputs):
    x = tf.nn.avg_pool(inputs, ksize=(7, 7), strides=1, padding='SAME')
    x = flatten_layer(x)
    x = tf.compat.v1.layers.dense(x, units=128)
    x = tf.nn.relu6(x)
    x = tf.nn.dropout(x, rate=0.5)
    x = tf.compat.v1.layers.dense(x, 2)
    output = tf.nn.softmax(x)
    
    return output

In [10]:
grad_log = []

epochs = 10 

for epoch in range(epochs): 
    print(f'\nНачинаем эпоху {epoch}') 

    for step in range(int(len(data.train.labels)/batch_size)):
        
        with tf.GradientTape() as tape: 
            
            x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)
            y_true_batch = tf.constant(y_true_batch)
            
            
            print(x_batch.shape)
            
            preds = tf.constant(trashcan_det(x_batch))
            print(type(preds))
            print(type(y_true_batch))
            print(preds)
            print(y_true_batch)
            
            loss_value = tf.nn.sigmoid_cross_entropy_with_logits(y_true_batch, preds)
            
            grads = tape.gradient(loss_value, trashcan_net.trainable_weights) 
            g_g = []

            # пишем логи для сохранения значений градиента и веса по одной цепи 
            for g_s in grads:
                # допишем логи значений градиента в зависимости от размера тензора градиента
                # if len(g_s.numpy().shape) == 1:
                #     g_g.append(g_s.numpy()[0])
                if len(g_s.numpy().shape) == 2:
                    g_g.append(g_s.numpy()[0, 0]) 

                
        # добавляем текущие логи по слоям к общей записи
        grad_log.append(g_g)

        # Выполним один шаг градиентного спуска,
        # обновив значение переменных минимизирующих потери
        optimizer.apply_gradients(zip(grads, trashcan_net.trainable_weights)) 

        # Пишем лог каждые 200 шагов
        if step % 200 == 0:
            print(f'Эпоха {epoch + 1}/{epochs}', end='. ')
            print(f'Шаг {step}. Лосс на обучении (для одного батча) на шаге: {loss_value}') 
            print(f'Уже увидели: {(step + 1) * batch_size} примеров')


Начинаем эпоху 0
(32, 244, 244, 3)
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[[0.14207473 0.8579253 ]
 [0.38127396 0.6187261 ]
 [0.16056512 0.83943486]
 [0.04831333 0.9516867 ]
 [0.1056706  0.8943294 ]
 [0.09794291 0.9020571 ]
 [0.26298803 0.737012  ]
 [0.04095499 0.95904493]
 [0.2794373  0.7205627 ]
 [0.06018573 0.9398142 ]
 [0.2882664  0.71173364]
 [0.02160314 0.97839683]
 [0.22157043 0.7784296 ]
 [0.06638856 0.93361145]
 [0.15895443 0.84104556]
 [0.20124657 0.7987534 ]
 [0.03361719 0.96638286]
 [0.24401195 0.755988  ]
 [0.1860317  0.81396836]
 [0.03866158 0.9613384 ]
 [0.4317653  0.5682347 ]
 [0.00745654 0.99254346]
 [0.13011967 0.8698803 ]
 [0.05122506 0.94877493]
 [0.0431484  0.9568516 ]
 [0.03056427 0.96943575]
 [0.15006253 0.8499375 ]
 [0.1157185  0.88428146]
 [0.08678949 0.9132106 ]
 [0.08612431 0.91387576]
 [0.52277106 0.4772289 ]
 [0.01239006 0.98761   ]], shape=(32, 2), dtype=float32)
tf.Tensor(
[[

InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:Mul]